# Notebook to overplot MIRI photometry on existing PROSPECTOR fits

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import prospect.io.read_results as reader
import pickle as pkl
import pandas as pd

from astropy.io import fits
from astropy import units as u
from prospect.io.read_results import results_from
from astropy.table import Table
from prospector_utils.plotting import reconstruct, load_and_display, create_hist
from prospector_utils.analysis import compute_residuals
import prospect
print(prospect.__file__)

Set the galaxy_ids array

In [ ]:
table_path = '/Users/benjamincollins/University/Master/Red_Cardinal/photometry/phot_tables/Photometry_Table_MIRI_v6.fits'

table = Table.read(table_path, format='fits')
galaxy_ids = np.asarray([str(gid) for gid in table['ID']])

galaxy_ids = reversed(galaxy_ids)


Reconstruct and plot the prospector outputs with MIRI

In [ ]:
plot = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/fits/'
plot_nodust = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/fits_nodust/'

stats = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/pickle_files/'
stats_nodust = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/pickle_files_nodust_v2/'

for gid in galaxy_ids:
    reconstruct(int(gid), stats_dir=stats, add_duste=True)
    #reconstruct(int(gid), plot_dir=plot_nodust, stats_dir=stats_nodust, add_duste=False)
    
#reconstruct(12513, plot_dir=plot_dir, stats_dir=stats_dir, add_duste=False)
#reconstruct(16424, plot_dir=plot_dir, stats_dir=stats_dir, add_duste=False)
#reconstruct(9871, plot_dir=plot_dir, stats_dir=stats_dir, add_duste=False)
#reconstruct(12717, plot_dir=plot, stats_dir=stats, add_duste=True)
#reconstruct(12717, plot_dir=plot_nodust, stats_dir=stats_nodust, add_duste=False)


Section just to load pickle file and display the plot

In [ ]:
load_and_display(12717)
load_and_display(18769)

Check the stored obs file in pickle:

In [ ]:
table_path = '/Users/benjamincollins/University/Master/Red_Cardinal/photometry/phot_tables/Photometry_Table_MIRI_v6.fits'

table = Table.read(table_path, format='fits')
galaxy_ids = np.asarray([str(gid) for gid in table['ID']])

galaxy_ids = reversed(galaxy_ids)


all_rows = []

for objid in galaxy_ids:
    print(f"Processing galaxy ID: {objid}")
    galaxy_rows = compute_residuals(objid, show_plot=True)
    if galaxy_rows is None:
        continue
    else:
        all_rows.extend(galaxy_rows)  # concatenate lists

df = pd.DataFrame(all_rows)

os.makedirs('/Users/benjamincollins/University/Master/Red_Cardinal/prospector/analysis', exist_ok=True)
df.to_csv('/Users/benjamincollins/University/Master/Red_Cardinal/prospector/analysis/residuals.csv', index=False)

Now let's create the histograms

In [ ]:
csv_path = '/Users/benjamincollins/University/Master/Red_Cardinal/prospector/analysis/residuals.csv'
hist_dir = '/Users/benjamincollins/University/Master/Red_Cardinal/prospector/histograms/'

create_hist(csv_path, out_dir=hist_dir)

In [ ]:
npy_file = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/obs/obs_12717.npy'
obs = np.load(npy_file, allow_pickle=True).item()

param_file = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/params/params_MAP_12717.pkl'
params = pkl.load(open(param_file, 'rb'))
#print(params.keys())

param_file = '/Users/benjamincollins/University/master/Red_Cardinal/prospector/spec_calib/spec_calibrated_12717.pkl'
data = pkl.load(open(param_file, 'rb'))

print(data.keys())
print(data['emi_off']['MAP'].keys())

obs_file = f'/Users/benjamincollins/University/master/Red_Cardinal/prospector/obs/obs_12717.npy'
obs = np.load(obs_file, allow_pickle=True).item()
print(obs.keys())
print(len(obs['mask']))
print(len(obs['spectrum']))
print(len(obs['spectrum'][obs['mask']]))

plt.plot(obs['wavelength'][obs['mask']], obs['spectrum'][obs['mask']], label="Observed spectrum")
#plt.plot(data['wave_obs'], data['MAP']['wave_obs'], label="MAP spectrum")
#plt.plot(data['wave_obs'], data['nodust'], label="No dust", linestyle="--")
plt.plot(data['wave_obs'], data['emi_off']['MAP']['wave_obs'], label="No emission lines", linestyle="-")
plt.loglog()
plt.xlabel("Observed wavelength [Å]")
plt.ylabel("Flux [maggies or arbitrary units]")
plt.legend()
plt.title("Calibrated Model Spectrum Variants")
plt.show()